# 3.2 Linear Regression Implementation from Scratch

## Exercises

### Q1. What would happen if we were to initialize the weights to zero. Would the algorithm still work?

### Q2. Assume that you are Georg Simon Ohm trying to come up with a model between voltage and current. Can you use auto differentiation to learn the parameters of your model?

### Q3. Can you use Planckʼs Law to determine the temperature of an object using spectral energy density?

### Q4. What are the problems you might encounter if you wanted to compute the second derivatives? How would  you fix them?

### Q5. Why is the `reshape` function needed in the `squared_loss` function?

### Q6. Experiment using different learning rates to find out how fast the loss function value drops.

### Q7. If the number of examples cannot be divided by the batch size, what happens to the `data_iter` functionʼs behavior?